# Circularity

In [7]:
from matplotlib import pyplot as plt
import numpy as np
from auriga.images import figure_setup, set_axs_configuration
from auriga.snapshot import Snapshot
from auriga.settings import Settings

In [2]:
figure_setup()

In [3]:
def load_subhalo_star_circ(simulation: str) -> np.ndarray:
    """
    This method returns the circularity of the stars in the main subhalo for
    a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation to load.

    Returns
    -------
    np.ndarray
        The circularity.
    """
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    return s.circularity[(s.type == 4) \
        & (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)]

In [4]:
def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
    """
    This method adds a panel with the results for a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation.
    ax : plt.Axes
        The ax to which to add the plot.
    """

    circularity = load_subhalo_star_circ(simulation=simulation)
    ax.hist(x=circularity,
            bins=150,
            range=(-2.0, 2.0),
            histtype="step",
            density=True,
            label=r"$\texttt{" + simulation.upper() + "}$")
    ax.text(x=0.05,
            y=0.95,
            s=r"$\texttt{" + simulation.upper() + "}$",
            size=6.0, transform=ax.transAxes,
            ha='left', va='top',
            )

In [9]:
def plot_age_metal_rel_for_sample(simulations: list,
                                  filename: str):
    """
    This method creates a plot of the circularity distribution for the
    stars in the main object.

    Parameters
    ----------
    simulations : str
        A list of simulations to plot.
    filename : str
        The name of the output file.
    """

    n_simulations = len(simulations)

    fig = plt.figure(figsize=(7.2, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    set_axs_configuration(
        xlim=(-2, 2), ylim=(0, 2),
        xticks=[-1, 0, 1], yticks=[0, 0.5, 1, 1.5],
        xlabel="$\epsilon$", ylabel="PDF",
        axs=axs, n_used=n_simulations)

    for idx, ax in enumerate(axs.flat):
        ax.grid(True, ls='-', lw=0.25, c='silver')
        if idx < n_simulations:
            add_panel_for_galaxy(simulations[idx], ax=ax)
        else:
            ax.axis("off")

    for ext in ["pdf", "png"]:
        fig.savefig(f"../images/circularity/{filename}.{ext}")
    plt.close(fig)

In [8]:
# Create plots for all galaxies in two figures
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
plot_age_metal_rel_for_sample(simulations=originals,
                              filename="originals_s127")
plot_age_metal_rel_for_sample(simulations=reruns,
                              filename="reruns_s251")